# Inference 

## Setup

In [ ]:
# Check the GPU
!nvidia-smi

In [ ]:
# Install dependencies.
!pip install bitsandbytes transformers accelerate peft -q

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git -q

ESRGAN requirements

In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

In [ ]:
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -O RealESRGAN_x4plus.pth

Extracting the weights stored in ``bajirao_LoRA.zip`` file.

In [ ]:
!unzip bajirao_LORA

Logging into Huggingface.

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL
from huggingface_hub import login

login()


In [ ]:
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

In [ ]:
pipe.load_lora_weights("/content/bajirao_LORA_1500")
_ = pipe.to("cuda")

In [ ]:
# enter the prompt here
prompt = """In this epic scene, sks Bajirao rides a majestic white horse, exuding an aura of power and
nobility. He is adorned in ornate armor, featuring intricate designs and a prominent lion
emblem, symbolizing strength and courage. The background is a grand, ornate palace with
domed structures and elaborate architecture, bathed in warm, golden light that suggests
a setting sun."""

image = pipe(prompt=prompt, num_inference_steps=50).images[0]

In [ ]:
image

In [ ]:
image.save("bajirao_image.png")

In [ ]:
from RealESRGAN import RealESRGAN
import os

device = 'cuda'
ESR_model = RealESRGAN(device, scale=4)
ESR_model.load_weights(os.path.abspath("RealESRGAN_x4plus.pth"))
sr_image = ESR_model.predict(image)

In [ ]:
sr_image

In [ ]:
sr_image.save("/content/bajirao_image_sr.png")

## Code for performing bicubic and Lanczos upsampling

In [ ]:
from PIL import Image
import numpy as np
import cv2

cv_image = np.array(image)

cv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2BGR)

target_size = (4096, 4096)

bicubic_upscaled = cv2.resize(cv_image, target_size, interpolation=cv2.INTER_CUBIC)

lanczos_upscaled = cv2.resize(cv_image, target_size, interpolation=cv2.INTER_LANCZOS4)

bicubic_pil = Image.fromarray(cv2.cvtColor(bicubic_upscaled, cv2.COLOR_BGR2RGB))
lanczos_pil = Image.fromarray(cv2.cvtColor(lanczos_upscaled, cv2.COLOR_BGR2RGB))

bicubic_pil.save("bajirao_image_bicubic.png")
lanczos_pil.save("bajirao_image_lanczos.png")